# Final Project
In this project we are building a spark notes generator.  In the event that I find myself a little crunched for time when reading a book and need necessary details rapidly, this program will *theoretically* take a few words off of my plate.  

This project will showcase my skill with: webscraping, regular expressions, and scoring.  Although web scraping was certianly not a part of this course, the ends are highly necessary to have an option to read in data from most places, if it is acceptable to scape the page.  Regular expression were talked about extensivly in this course and is used to pick up speach patterns.  




In [1]:
import bs4 as bs
import urllib.request
import re
import nltk

## Importing the data
Using the most common webscraping format, we import the data based off of a url and pick up the paragraphs.  I have tested this for limited sites, and if crucial information is in headdings or other htypes of tags, it will get missed.  This did not have to get built into a function, but I would like to keep this handy for reusablility.   

In [32]:
def get_data(url):
#PRE: beautiful soup and nltk are imported
    scraped = urllib.request.urlopen(url).read()
    parsed =  bs.BeautifulSoup(scraped, 'lxml')
    paragraphs = parsed.find_all('p')
    text = ""
    for p in paragraphs:
        text += p.text
    #tokens = nltk.sent_tokenize(text)
    return text

In [40]:
data = get_data('https://www.nltk.org/book/ch01.html')

# Preprocessing
Much like any Data science project, just because we have the data, does not mean that it does not require any further refinement.  With the scraped data we hope to reduce the text to readable messages, tokenize the data, and rule out stopwords.
## Removing texts 
Below we are replcing numbers, duplicate whitespace, and non-alpha's then turns them into whitespace. 

In [41]:
data = re.sub(r'\[[0-9]*\]', ' ', data)
data = re.sub(r'\s+', ' ', data)
data = re.sub('[^a-zA-Z]', ' ', data)
data = re.sub(r'\s+', ' ', data)

## Tokenize sentance
Below we tokenize the individual sentances into their own object. 

In [51]:
tokens = nltk.sent_tokenize(data)

## Implementing Stopwords 
Instead of brainstorming my own set of stopwords, like I have in the past, I am simply poaching the nltk coprus collection.  This is a list of words that is far more elaborate than what I could come up with and is a hundred times easier to develop.  This list consists of 179 words that are all too common in speech to see recognize a signigificant use.  

The reason that eliminating stopwords is so important in this project is that our stratagy is dependant on the word frequency in each sentance, and using common words would cause an unnecessary noise that the model would not be able to see through.  

In [43]:
stopwords = nltk.corpus.stopwords.words('english')

## Word Frequency
Below we fufil our strategy of getting the word frequency of each word in the sentances.  We include the word frequncy because theoretically, if a word is used more frequently, the sentance it pertains to is more important.  Thus we want to iterate through the article to find the most freqently used words.

In [44]:
def get_word_frequency(formatted_article):
    word_frequency = {}
    for word in nltk.word_tokenize(formatted_article):
        if word not in stopwords:
            if word not in word_frequency.keys():
                word_frequency[word] = 1
            else: 
                word_frequency[word] += 1
    return word_frequency

In [47]:
word_frequncy = get_word_frequency(data)

## Sentance Score
Similar to above, we need to iterate through the entire text.  Utilizing the frequncy from above, we sum them into the score below and  get the relevance score of all of the data.  

In [59]:
def get_sentance_score(tokenize_sentance, freq):
    score = {}
    for sent in tokenize_sentence:
        for word in nltk.word_tokenize(sent.lower()):
            if word in freq.keys():
                if len(sent.split(' ')) < 50:
                    if sent not in score.keys():
                        score[sent] = freq[word]
                    else:
                        score[sent] += freq[word]
    return score

In [60]:
sentance_score = get_sentance_score(tokens, word_frequency)

# Output the sparknotes
Finally, we have built the scores for getting the most important information.  Now using the heapq package we can output them in a priority queue with the most important data.  

In [56]:
import heapq 
sentence_summary = heapq.nlargest(30, sentence_score, key = sentence_score.get)
summary = ' '.join(sentence_summary)
print(summary)

The next example shows us that the number of distinct words is just 6% of the total number of words, or equivalently that each word is used 16 times on average (remember if you're using Python 2, to start with from __future__ import division).Next, let's focus on particular words. These 50 words account for nearly half the book!Figure 3.2: Cumulative Frequency Plot for 50 Most Frequently Words in Moby Dick: these account for nearly half of the tokens.If the frequent words don't help us, how about the words that occur once only, the so-called hapaxes? One might wonder how frequent the different lengths of word are (e.g., how many words of length four appear in the text, are there more words of length five than length four, etc). We can count how often a word occurs in a text, and compute what percentage of the text is taken up by a specific word:NoteYour Turn: How many times does the word lol appear in text5? Does it make a difference to your results if you change the variable names, e.